In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# -----------------------------
# CONFIGURATION
# -----------------------------
DATASET_DIR = r"C:\Users\USER\Desktop\DATASETS\XRAY"
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 12        # 10–15 epochs recommended
MODEL_NAME = "resnet"     # choose: "resnet" or "mobilenet"
SAVE_MODEL_PATH = "classifier.h5"

# -----------------------------
# DATA GENERATORS
# -----------------------------
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1/255.0)
test_datagen = ImageDataGenerator(rescale=1/255.0)

train_gen = train_datagen.flow_from_directory(
    os.path.join(DATASET_DIR, "train"),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_gen = val_datagen.flow_from_directory(
    os.path.join(DATASET_DIR, "val"),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)arget_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

num_classes = len(train_gen.class_indices)
print("Classes detected:", train_gen.class_indices)

# -----------------------------
# CHOOSE BASE MODEL
# -----------------------------
if MODEL_NAME == "resnet":
    base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
elif MODEL_NAME == "mobilenet":
    base_model = MobileNetV2(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
else:
    raise ValueError("MODEL_NAME must be 'resnet' or 'mobilenet'")

# Freeze pretrained layers
base_model.trainable = False

# -----------------------------
# BUILD CLASSIFIER HEAD
# -----------------------------
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# -----------------------------
# COMPILE MODEL
# -----------------------------
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# -----------------------------
# TRAIN
# -----------------------------
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS
)

# -----------------------------
# EVALUATE ON TEST SET
# -----------------------------
test_loss, test_acc = model.evaluate(test_gen)
print(f"\nTest Accuracy: {test_acc:.4f}")

# -----------------------------
# SAVE MODEL
# -----------------------------
model.save(SAVE_MODEL_PATH)
print(f"\nModel saved successfully as: {SAVE_MODEL_PATH}")


Found 6326 images belonging to 4 classes.
Found 38 images belonging to 4 classes.
Found 771 images belonging to 4 classes.
Classes detected: {'COVID19': 0, 'NORMAL': 1, 'PNEUMONIA': 2, 'TURBERCULOSIS': 3}
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 22s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,113,284 (91.98 MB)

 Trainable params: 525,572 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/12
198/198 ━━━━━━━━━━━━━━━━━━━━ 197s 982ms/step - accuracy: 0.5827 - loss: 1.1343 - val_accuracy: 0.2105 - val_loss: 1.6301
Epoch 2/12
198/198 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.6140 - loss: 1.0319 - val_accuracy: 0.2368 - val_loss: 1.4603
Epoch 3/12
198/198 ━━━━━━━━━━━━━━━━━━━━ 174s 879ms/step - accuracy: 0.6209 - loss: 1.0111 - val_accuracy: 0.2368 - val_loss: 1.4444
Epoch 4/12
198/198 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.6261 - loss: 0.9814 - val_accuracy: 0.2632 - val_loss: 1.5580
Epoch 5/12
198/198 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.6334 - loss: 0.9585 - val_accuracy: 0.2368 - val_loss: 1.3425
Epoch 6/12
198/198 ━━━━━━━━━━━━━━━━━━━━ 199s 1s/step - accuracy: 0.6394 - loss: 0.9428 - val_accuracy: 0.2632 - val_loss: 1.4085
Epoch 7/12
198/198 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.6420 - loss: 0.9261 - val_accuracy: 0.2895 - val_loss: 1.4236
Epoch 8/12
198/198 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.6476 - loss: 0.9001 - va


Test Accuracy: 0.6368

Model saved successfully as: classifier.h5
